In [1]:
# This makes a first contrast curve for Altair

# created 2018 Sept. 20 by E.S.

## SECTION TO INITIALIZE

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage

In [2]:
# import stuff

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule, \
                                            AttributeReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ContrastCurveModule, \
                                       FluxAndPosition, \
                                    StackingAndSubsampling
from PynPoint.ProcessingModules.StarAlignment import StarCenteringModule
#from PynPoint.ProcessingModules.StackingAndSubsampling import DerotateAndStackModule
from PynPoint.ProcessingModules.FluxAndPosition import FakePlanetModule
#from PynPoint.Util import AnalysisTools

ImportError: cannot import name scimath

In [2]:
import os
from astropy.io import fits
from astropy.modeling import models, fitting

In [1]:
# stem 

stem_base = ('/home/../../media/unasemaje/Elements/lbti_data_reduction/180507_fizeau_altair/')
stem_retrieve = (stem_base+'03_ramp_removed_pca_background_subted/')
stem_place = (stem_base+'04_gross_cookie_cutout/')
stem_place_centered = (stem_base+'05_centered/')
stem_place_fftinfo = (stem_base+'05b_fftinfo/')

# Make PSF cutouts

In [3]:
# Make cookie cut-outs of Fizeau PSFs (note the centering will not be great)
# retrieve from /03_ramp_removed_pca_background_subted/
# deposit in /04_gross_cookie_cutout/

In [11]:
def make_cutout(framenum):
    '''
    Write out cut-outs of PSFs
    '''
    
    img_string = 'lm_180507_'+str("{:0>6d}".format(framenum))+'_nPCA164.fits'
    
    # if file exists
    if os.path.isfile(stem_retrieve+img_string):
        
        # read in image
        sciImg, header = fits.getdata(stem_retrieve+img_string,0,header=True)
        
        if (framenum < 7735):
            psf_loc = [310,850] # (y,x) convention; for frames <7735, where the PSF is in the up nod position
        elif (framenum > 7772):
            psf_loc = [115,865] # (y,x) convention; for frames >7772, where the PSF is in the down nod position
        else:
            print("Frame number not legit!")
        
        # write a cookie-cut frame back out
        hdu = fits.PrimaryHDU(sciImg[psf_loc[0]-50:psf_loc[0]+50,
                                     psf_loc[1]-50:psf_loc[1]+50].astype(np.float64), 
                              header=header)
        hdul = fits.HDUList([hdu])
    
        hdul.writeto(stem_place+img_string, overwrite=False)
        #print('Saved '+str("{:0>6d}".format(framenum)))
        
    else:
        
        print('File '+img_string+' not found')

In [12]:
# make the cutouts

for frameNum in range(4249,11336):
    make_cutout(frameNum)

Saved 004249
Saved 004250
Saved 004251
Saved 004252
Saved 004253
Saved 004254
Saved 004255
Saved 004256
Saved 004257
Saved 004258
Saved 004259
Saved 004260
Saved 004261
Saved 004262
Saved 004263
Saved 004264
Saved 004265
Saved 004266
Saved 004267
Saved 004268
Saved 004269
Saved 004270
Saved 004271
Saved 004272
Saved 004273
Saved 004274
Saved 004275
Saved 004276
Saved 004277
Saved 004278
Saved 004279
Saved 004280
Saved 004281
Saved 004282
Saved 004283
Saved 004284
Saved 004285
Saved 004286
Saved 004287
Saved 004288
Saved 004289
Saved 004290
Saved 004291
Saved 004292
Saved 004293
Saved 004294
Saved 004295
Saved 004296
Saved 004297
Saved 004298
Saved 004299
Saved 004300
Saved 004301
Saved 004302
Saved 004303
Saved 004304
Saved 004305
Saved 004306
Saved 004307
Saved 004308
Saved 004309
Saved 004310
Saved 004311
Saved 004312
Saved 004313
Saved 004314
Saved 004315
Saved 004316
Saved 004317
Saved 004318
Saved 004319
Saved 004320
Saved 004321
Saved 004322
Saved 004323
Saved 004324
Saved 004325

# Define workspaces for PynPoint

In [4]:
# define workspaces and initialize Pypeline

working_place = stem_base+"./pynpoint_testing/pynpoint_experimentation_altair/working_place/nov21_working/"
input_place = stem_place
output_place = stem_base+"./pynpoint_testing/pynpoint_experimentation_altair/output_place/nov21_output/"

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


In [5]:
## IF THE *INI FILE WAS NEWLY GENERATED, EDIT THE *INI FILE TO SET
## PIXSCALE = 0.0107

# confirm PIXSCALE change
'''
pixscale_config = pipeline.get_attribute("config", "PIXSCALE")
print("Plate scale for /config/ is "+pixscale_config)
pixscale_sci = pipeline.get_attribute("science", "PIXSCALE")
print("Plate scale for /science/ is "+pixscale_sci)
'''

'\npixscale_config = pipeline.get_attribute("config", "PIXSCALE")\nprint("Plate scale for /config/ is "+pixscale_config)\npixscale_sci = pipeline.get_attribute("science", "PIXSCALE")\nprint("Plate scale for /science/ is "+pixscale_sci)\n'

## READ IN DATA

In [5]:
# Read in all PSF cookie cut-outs from FITS files 

read_all_psfs = FitsReadingModule(name_in="read_all_psfs",
                                 input_dir=input_place,
                                 image_tag="all_psfs_precentering",
                                 check=True)

pipeline.add_module(read_all_psfs)

## CENTER PSFS

In [8]:
center_psfs = StarCenteringModule(name_in="center_psfs", 
                                              image_in_tag="all_psfs_precentering", 
                                              image_out_tag="all_psfs_centered", 
                                              mask_out_tag=None, 
                                              fit_out_tag="center_fit", 
                                              method="full", 
                                              interpolation="spline", 
                                              radius=0.16, 
                                              sign="positive")

pipeline.add_module(center_psfs)


# write cube of centered frames (no PA correction)
write_centered_psfs = FitsWritingModule(file_name="centering_test.fits",
                              name_in="write_centered_psfs",
                              output_dir=output_place,
                              data_tag="all_psfs_centered")
                              
pipeline.add_module(write_centered_psfs)

In [9]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:167: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:167: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:167: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:197: UserWarning: Non-static attribute PUPIL (=ESO ADA PUPILPOS) not found in the FITS header.
  "FITS header." % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:197: UserWarning: Non-static attribute DEC (=DE

Running FitsReadingModule... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Running StarCenteringModule... [DONE]
2D Gaussian fit could not converge on 982 image(s). [WARNING]
Running FitsWritingModule... [DONE]


In [ ]:
########################################################################################################################

# Now upsample and center the images again, without using PynPoint (requires Python 3.5 for plots)

In [25]:
#%matplotlib qt

In [4]:
# read in cutout frames

def center_cutouts(framenum):
    '''
    Center PSF cutouts without using PynPoint
    '''

    # fit 2D Gaussian
    img_string = 'lm_180507_'+str("{:0>6d}".format(framenum))+'_nPCA164.fits'
    
    # if file exists
    if os.path.isfile(stem_place+img_string):
        
        # read in image
        sciImg, header = fits.getdata(stem_place+img_string,0,header=True)
        
        # get coordinate grid info
        y, x = np.mgrid[0:np.shape(sciImg)[0],0:np.shape(sciImg)[1]]
        z = np.copy(sciImg)
        
        # make an initial Gaussian guess
        p_init = models.Gaussian2D(amplitude=60000.,
                                   x_mean=50.,
                                   y_mean=50.,
                                   x_stddev=6.,
                                   y_stddev=6.)
        fit_p = fitting.LevMarLSQFitter()

        # fit the data
        p = fit_p(p_init, x, y, z)
        ampl, x_mean, y_mean, x_stdev, y_stdev, theat = p._parameters
        
        # Plot the data with the best-fit model
        '''
        plt.clf()
        plt.figure(figsize=(8, 2.5))
        plt.subplot(1, 3, 1)
        plt.imshow(z, origin='lower', interpolation='nearest', vmin=-10, vmax=60000)
        plt.title("Data")
        plt.subplot(1, 3, 2)
        plt.imshow(p(x, y), origin='lower', interpolation='nearest', vmin=-10, vmax=60000)
        plt.title("Model")
        plt.subplot(1, 3, 3)
        plt.imshow(z - p(x, y), origin='lower', interpolation='nearest', vmin=-10, vmax=60000)
        plt.title("Residual")
        plt.suptitle("Frame "+str("{:0>6d}".format(framenum)))
        plt.savefig(stem_place_centered+'pngs_fits/png_fit_lm_180507_'+str("{:0>6d}".format(framenum))+'.png',overwrite=True)
        plt.close()
        '''
                
        # center the frame
        # N.b. for a 100x100 image, the physical center is at Python coordinate (49.5,49.5)
        # i.e., in between pixels 49 and 50 in both dimensions (Python convention),
        # or at coordinate (50.5,50.5) in DS9 convention
        sciImg_shifted = scipy.ndimage.interpolation.shift(sciImg, shift = [49.5-y_mean, 49.5-x_mean]) # shift in +y,+x convention
        
        # write a cookie-cut frame back out
        hdu = fits.PrimaryHDU(sciImg_shifted,header=header)
        hdul = fits.HDUList([hdu])
        hdul.writeto(stem_place_centered+'lm_180507_'+str("{:0>6d}".format(framenum))+'_nPCA164.fits', 
                     overwrite=True)
        
        print("Centered frame "+str("{:0>6d}".format(framenum)))

In [5]:
for frameNum in range(4650,11336):
    center_cutouts(frameNum)

Centered frame 004650
Centered frame 004651
Centered frame 004652
Centered frame 004653
Centered frame 004654
Centered frame 004655
Centered frame 004656
Centered frame 004657
Centered frame 004658
Centered frame 004659
Centered frame 004660
Centered frame 004661
Centered frame 004662
Centered frame 004663
Centered frame 004664
Centered frame 004665
Centered frame 004666
Centered frame 004667
Centered frame 004668
Centered frame 004669
Centered frame 004670
Centered frame 004671
Centered frame 004672
Centered frame 004673
Centered frame 004674
Centered frame 004675
Centered frame 004676
Centered frame 004677
Centered frame 004678
Centered frame 004679
Centered frame 004680
Centered frame 004681
Centered frame 004682
Centered frame 004683
Centered frame 004684
Centered frame 004685
Centered frame 004686
Centered frame 004687
Centered frame 004688
Centered frame 004689
Centered frame 004690
Centered frame 004691
Centered frame 004692
Centered frame 004693
Centered frame 004694
Centered f

Centered frame 005024
Centered frame 005025
Centered frame 005026
Centered frame 005027
Centered frame 005028
Centered frame 005029
Centered frame 005030
Centered frame 005031
Centered frame 005032
Centered frame 005033
Centered frame 005034
Centered frame 005035
Centered frame 005036
Centered frame 005037
Centered frame 005038
Centered frame 005039
Centered frame 005040
Centered frame 005041
Centered frame 005042
Centered frame 005043
Centered frame 005044
Centered frame 005045
Centered frame 005046
Centered frame 005047
Centered frame 005048
Centered frame 005049
Centered frame 005050
Centered frame 005051
Centered frame 005052
Centered frame 005053
Centered frame 005054
Centered frame 005055
Centered frame 005056
Centered frame 005057
Centered frame 005058
Centered frame 005059
Centered frame 005060
Centered frame 005061
Centered frame 005062
Centered frame 005063
Centered frame 005064
Centered frame 005065
Centered frame 005066
Centered frame 005067
Centered frame 005068
Centered f

Centered frame 005398
Centered frame 005399
Centered frame 005400
Centered frame 005401
Centered frame 005402
Centered frame 005403
Centered frame 005404
Centered frame 005405
Centered frame 005406
Centered frame 005407
Centered frame 005408
Centered frame 005409
Centered frame 005410
Centered frame 005411
Centered frame 005412
Centered frame 005413
Centered frame 005414
Centered frame 005415
Centered frame 005416
Centered frame 005417
Centered frame 005418
Centered frame 005419
Centered frame 005420
Centered frame 005421
Centered frame 005422
Centered frame 005423
Centered frame 005424
Centered frame 005425
Centered frame 005426
Centered frame 005427
Centered frame 005428
Centered frame 005429
Centered frame 005430
Centered frame 005431
Centered frame 005432
Centered frame 005433
Centered frame 005434
Centered frame 005435
Centered frame 005436
Centered frame 005437
Centered frame 005438
Centered frame 005439
Centered frame 005440
Centered frame 005441
Centered frame 005442
Centered f

Centered frame 005619
Centered frame 005623
Centered frame 005624
Centered frame 005625
Centered frame 005626
Centered frame 005628
Centered frame 005630
Centered frame 005633
Centered frame 005634
Centered frame 005635
Centered frame 005636
Centered frame 005638
Centered frame 005639
Centered frame 005640
Centered frame 005641
Centered frame 005642
Centered frame 005643
Centered frame 005644
Centered frame 005645
Centered frame 005646
Centered frame 005647
Centered frame 005648
Centered frame 005649
Centered frame 005650
Centered frame 005651
Centered frame 005652
Centered frame 005653
Centered frame 005654
Centered frame 005655
Centered frame 005656
Centered frame 005657
Centered frame 005658
Centered frame 005659
Centered frame 005660
Centered frame 005661
Centered frame 005662
Centered frame 005663
Centered frame 005664
Centered frame 005665
Centered frame 005666
Centered frame 005667
Centered frame 005668
Centered frame 005669
Centered frame 005670
Centered frame 005671
Centered f

Centered frame 006005
Centered frame 006006
Centered frame 006007
Centered frame 006008
Centered frame 006009
Centered frame 006010
Centered frame 006011
Centered frame 006012
Centered frame 006013
Centered frame 006014
Centered frame 006015
Centered frame 006016
Centered frame 006017
Centered frame 006018
Centered frame 006019
Centered frame 006020
Centered frame 006021
Centered frame 006022
Centered frame 006023
Centered frame 006024
Centered frame 006025
Centered frame 006026
Centered frame 006027
Centered frame 006028
Centered frame 006029
Centered frame 006030
Centered frame 006031
Centered frame 006032
Centered frame 006033
Centered frame 006034
Centered frame 006035
Centered frame 006036
Centered frame 006037
Centered frame 006038
Centered frame 006039
Centered frame 006040
Centered frame 006041
Centered frame 006042
Centered frame 006043
Centered frame 006044
Centered frame 006045
Centered frame 006046
Centered frame 006047
Centered frame 006048
Centered frame 006049
Centered f

...

Centered frame 009825
Centered frame 009826
Centered frame 009827
Centered frame 009828
Centered frame 009829
Centered frame 009830
Centered frame 009831
Centered frame 009832
Centered frame 009833
Centered frame 009834
Centered frame 009835
Centered frame 009836
Centered frame 009837
Centered frame 009838
Centered frame 009839
Centered frame 009840
Centered frame 009841
Centered frame 009842
Centered frame 009843
Centered frame 009844
Centered frame 009845
Centered frame 009846
Centered frame 009847
Centered frame 009848
Centered frame 009849
Centered frame 009850
Centered frame 009851
Centered frame 009852
Centered frame 009853
Centered frame 009854
Centered frame 009855
Centered frame 009856
Centered frame 009857
Centered frame 009858
Centered frame 009859
Centered frame 009860
Centered frame 009861
Centered frame 009862
Centered frame 009863
Centered frame 009864
Centered frame 009865
Centered frame 009866
Centered frame 009867
Centered frame 009868
Centered frame 009869
Centered f

Centered frame 010201
Centered frame 010202
Centered frame 010203
Centered frame 010204
Centered frame 010205
Centered frame 010206
Centered frame 010207
Centered frame 010208
Centered frame 010209
Centered frame 010210
Centered frame 010211
Centered frame 010212
Centered frame 010213
Centered frame 010214
Centered frame 010215
Centered frame 010216
Centered frame 010217
Centered frame 010218
Centered frame 010219
Centered frame 010220
Centered frame 010221
Centered frame 010222
Centered frame 010223
Centered frame 010224
Centered frame 010225
Centered frame 010226
Centered frame 010227
Centered frame 010228
Centered frame 010229
Centered frame 010230
Centered frame 010231
Centered frame 010232
Centered frame 010233
Centered frame 010234
Centered frame 010235
Centered frame 010236
Centered frame 010237
Centered frame 010238
Centered frame 010239
Centered frame 010240
Centered frame 010241
Centered frame 010242
Centered frame 010243
Centered frame 010244
Centered frame 010245
Centered f

Centered frame 010574
Centered frame 010575
Centered frame 010576
Centered frame 010577
Centered frame 010578
Centered frame 010579
Centered frame 010580
Centered frame 010581
Centered frame 010582
Centered frame 010583
Centered frame 010584
Centered frame 010585
Centered frame 010586
Centered frame 010587
Centered frame 010588
Centered frame 010589
Centered frame 010590
Centered frame 010591
Centered frame 010592
Centered frame 010593
Centered frame 010594
Centered frame 010595
Centered frame 010596
Centered frame 010597
Centered frame 010598
Centered frame 010599
Centered frame 010600
Centered frame 010601
Centered frame 010602
Centered frame 010603
Centered frame 010604
Centered frame 010605
Centered frame 010606
Centered frame 010607
Centered frame 010608
Centered frame 010609
Centered frame 010610
Centered frame 010611
Centered frame 010612
Centered frame 010613
Centered frame 010614
Centered frame 010615
Centered frame 010616
Centered frame 010617
Centered frame 010618
Centered f

Centered frame 010949
Centered frame 010950
Centered frame 010951
Centered frame 010952
Centered frame 010953
Centered frame 010954
Centered frame 010955
Centered frame 010956
Centered frame 010957
Centered frame 010958
Centered frame 010959
Centered frame 010960
Centered frame 010961
Centered frame 010962
Centered frame 010963
Centered frame 010964
Centered frame 010965
Centered frame 010966
Centered frame 010967
Centered frame 010968
Centered frame 010969
Centered frame 010970
Centered frame 010971
Centered frame 010972
Centered frame 010973
Centered frame 010974
Centered frame 010975
Centered frame 010976
Centered frame 010977
Centered frame 010978
Centered frame 010979
Centered frame 010980
Centered frame 010981
Centered frame 010982
Centered frame 010983
Centered frame 010984
Centered frame 010985
Centered frame 010986
Centered frame 010987
Centered frame 010988
Centered frame 010989
Centered frame 010990
Centered frame 010991
Centered frame 010992
Centered frame 010993
Centered f

Centered frame 011325
Centered frame 011326
Centered frame 011327
Centered frame 011328
Centered frame 011329
Centered frame 011330
Centered frame 011331
Centered frame 011332
Centered frame 011333
Centered frame 011334
Centered frame 011335


# Take FFTs of the newly centered images, and write characteristics to a file by using testing_making_set_masks_copiedFromFizControl.ipynb(has to run in Python 2.7!!)